In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import seaborn as sns
import itertools
from itertools import groupby
import os
import re
from pathlib import Path
import ast

In [2]:
def assign_flags(gaze_df, events):
    """
    Given a gaze DataFrame and event intervals, assign flag and emotion_side.
    Returns the DataFrame with added 'flag' and 'emotion_side' columns.
    """
    # Initialize new columns.
    gaze_df['flag'] = -1
    gaze_df['emotion_side'] = "N"  # Default to "N" if no emotion side is applicable.
    
    # Apply flag 0 for fixation intervals.
    for start, stop in events['fixation']:
        mask = (gaze_df['Time'] >= float(start)) & (gaze_df['Time'] <= float(stop))
        gaze_df.loc[mask, 'flag'] = 0
    
    # Apply flag 1 for faces stimuli intervals and assign emotion.
    for start, stop, emotion in events['fractals']:
        mask = (gaze_df['Time'] >= float(start)) & (gaze_df['Time'] <= float(stop))
        gaze_df.loc[mask, 'flag'] = 1
        gaze_df.loc[mask, 'emotion_side'] = emotion  # "L", "R", or "N" if not captured.
    
    # Apply flag 2 for pause intervals.
    for start, stop in events['pause']:
        mask = (gaze_df['Time'] >= float(start)) & (gaze_df['Time'] <= float(stop))
        gaze_df.loc[mask, 'flag'] = 2
    
    return gaze_df

In [ ]:
pathlist = Path('../../asc_data').rglob('*.asc')
task_data_df = []
df_final = pd.DataFrame()
time_delay_non_cued = []
placement_position_saccade = []
# for all participant
for path in pathlist:
    participant= re.search(r"(P\d+)", os.path.basename(str(path.stem))).group(1)
    # if participant == 'P157':
    f = open(str(path), 'r') # 'r' = read
    lines = f.read()
    f.close()
    # j = 4
    # n = 30
    indices = []
    current_start_index = 0
    not_list = []
    # for i in range(1):
    index_init1 = re.search(r'Dot Probe Fractals Experiemnt Starts',lines[current_start_index:-1]).span()[1]
    index_init2 = re.search(r'\n',lines[current_start_index + index_init1:-1]).span()[1]
    index_init =  current_start_index + index_init1 + index_init2

    index_finit1 = re.search(r'Dot Probe Fractals Experiment Ends',lines[index_init:-1]).span()[0]
    index_finit = index_init + index_finit1

    indices.append([index_init, index_finit-1])
    current_start_index = index_finit

    # if(i + 1) % 5 == 0:
    #     j = int(j/2)

    new_vec = lines[index_init: index_finit-1].split("\n")
    new_new_vec = []
    #0 = blink, 1 fixation, 2 saccade
    stateL = 0
    stateR = 0

    events = {
        # 'experiment': None,
        'fixation': [],
        'fractals': [],
        'pause': []
    }
    fixation_start = None
    fractals_start = None
    fractals_emotion = None
    pause_start = None
    exp_start = None
    exp_end = None

    # print('....................',len(new_vec))
    for k in range(1,len(new_vec)):
        la = new_vec[k].split("\t")
        if len(la) == 1:
            if 'SSACC L' in la[0]:
                stateL = 2
            if 'SFIX L' in la[0]:
                stateL = 1
            if 'SBLINK L' in la[0]:
                stateL = 0
            if 'SSACC R' in la[0]:
                stateR = 2
            if 'SFIX R' in la[0]:
                stateR = 1
            if 'SBLINK R' in la[0]:
                stateR = 0

        if len(la) == 2:
            ts = la[1].split(' ')[0]
            if 'Fixation_Cross_Start' in la[1]:
                fixation_start = ts
            elif 'Fixation_Cross_Stop' in la[1] and fixation_start is not None:
                events['fixation'].append((fixation_start, ts))
                fixation_start = None

            if 'Fractals_Stimuli_Start' in la[1]:
                fractals_start = ts
                fractals_emotion = None
            
            if 'Fractal Side' in la[1]:
                m = re.search(r"fractal side:\s*(\w+)", la[1], re.IGNORECASE)
                if m:
                    emo = m.group(1).lower()
                    fractals_emotion = "L" if emo == "left" else "R"

            elif 'Fractals_Stimuli_Stop' in la[1] and fractals_start is not None: 
                if fractals_emotion is None:
                    fractals_emotion = "N"
                events['fractals'].append((fractals_start, ts, fractals_emotion))
                fractals_start = None
                fractals_emotion = None

            if 'Pause_Start' in la[1]:
                pause_start = ts
            elif 'Pause_Stop' in la[1] and pause_start is not None:
                events['pause'].append((pause_start, ts))
                pause_start = None

        if len(la) == 9:
            if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                continue
            else:
                la[-1] = stateL
                la.append(stateR)
                new_new_vec.append(la)
        if len(la) == 6:
            if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                continue
            else:
                la[-1] = stateL
                la.append(stateR)
                new_new_vec.append(la)

    with open("new_new_vec.txt", "w") as file:
        for item in new_new_vec:
            file.write(str(item) + "\n")

    # print(events)

    ls = [[np.nan if x == '   .' else x for x in y] for y in new_new_vec]
    if len(ls)>0:
        ls = np.swapaxes(ls,0,1)
        current_trial = np.asarray(ls, dtype=float)
        # if participant =='P205':
        # print(len(current_trial))
        if len(current_trial) == 10:
            my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[4][:],current_trial[5][:],current_trial[-1][:]]),0,1)), columns=['Time','RX','RY','state'])
        else:
            # print(current_trial[7][:])
            my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[1][:],current_trial[2][:],current_trial[5][:]]),0,1)), columns=['Time','RX','RY','state'])
        my_df['participants'] = participant
        assign_flags(my_df, events)

        task_data_df.append(my_df)
        df_final = pd.concat([df_final,my_df], ignore_index=True)
           
                
df_final.to_csv("../../processed_data/processed_data_asc_task_fractals_all.csv", index=False)
    

In [6]:
df_final

,Time,RX,RY,state,participants,flag,emotion_side
0,71345603.0,991.3,482.2,0.0,P157,-1,N
1,71345604.0,990.9,482.3,0.0,P157,-1,N
2,71345605.0,990.5,481.1,0.0,P157,0,N
3,71345606.0,990.6,479.8,0.0,P157,0,N
4,71345607.0,990.6,479.8,0.0,P157,0,N
...,...,...,...,...,...,...,...
341389,12342603.0,1470.6,615.2,1.0,P205,1,L
341390,12342604.0,1470.9,613.3,1.0,P205,1,L
341391,12342605.0,1471.2,611.3,1.0,P205,1,L
341392,12342606.0,1471.2,611.2,1.0,P205,1,L
